TypeError: ApiDataSaver.saving_at_s3() missing 1 required positional argument: 'weather_json_string'

In [1]:
import os
import json
import boto3
import requests

In [12]:
profile_name='CRISTIAN_AWS'
region_name='us-east-1'


client = boto3.session.Session(profile_name=profile_name).client('secretsmanager', region_name)
response = client.get_secret_value(SecretId='env/openweather')
secret_data = response['SecretString']
secrets_dict = json.loads(secret_data)
open_weather_token = secrets_dict.get('OPEN_WEATHER_SECRET', '')
API_LINK = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID={open_weather_token}"

In [13]:
requisicao = requests.get(API_LINK)
wheater_json = requisicao.json()
weather_json_string = json.dumps(wheater_json)

In [16]:
s3 = boto3.session.Session(profile_name=profile_name).client('s3')
bucket_name = 'data-integration-projects'
directory_path = 'openweather-data-lake/raw-data/'
current_date = datetime.now()
dia_da_execucao = current_date.strftime('%Y-%m-%d-%H-%M-%S')
year = str(current_date.year)
month = str(current_date.month).zfill(2)  # Adicionar zero à esquerda se necessário
day = str(current_date.day).zfill(2)  # Adicionar zero à esquerda se necessário
directory_path = f'{directory_path}{year}/{month}/{day}/'
file_name = f'weather_data_{dia_da_execucao}.json'
s3.put_object(Bucket=bucket_name, Key=directory_path + file_name, Body=weather_json_string)
print(f'Arquivo salvo em: {bucket_name}/{directory_path}{file_name}')

Arquivo salvo em: data-integration-projects/openweather-data-lake/raw-data/2024/01/12/weather_data_2024-01-12-22-34-09.json
